# Part I. Creating an ETL Pipeline for Pre-Processing the Files

#### Importing Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating a list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Getting current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Created a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob('/home/workspace/event_data/*.csv')
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# Total number of rows 
#print(len(full_data_rows_list))
# The list of event data rows will look like
#print(full_data_rows_list)

# creating a event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


<br>

<br>

# Part II. Completing the Apache Cassandra coding portion of your project. 

### A CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory is created. It contains the following columns: 
* artist 
* firstName of user
* gender of user
* item number in session
* last name of user
* length of the song
* level (paid or free song)
* location of the user
* sessionId
* song title
* userId

The image below is a sample of the denormalized data:<br>

<img src="images/image_event_datafile_new.jpg">

<br>

<br>

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

<br>

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    {'class':'SimpleStrategy','replication_factor':1}
    """)
except Exception as e:
    print('ERROR!!')
    print(e)

<br>

#### Setting a Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print('ERROR!!')
    print(e)

<br>

<br>

## Query 1

> Give the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

For this query,the combination of sessionId and itemInSession is considered to be the composite key, because the query is based on these two attributes. Here, sessionID acts as the partition key and itemInSession is the clustering key. 
The attributes of the table are - ```sessionId, itemInSession, artist, song, length```

In [8]:
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS song_playlist_session
    (sessionId INT, itemInSession INT, artist TEXT, song TEXT, length FLOAT, PRIMARY KEY(sessionId,itemInSession) );
    """)
except Exception as e:
    print('ERROR!!')
    print(e)                    

In [9]:
# Code to set up the CSV file.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_session (sessionId , itemInSession , artist , song , length )"
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### SELECT Query

So according to the query, we require artist artist, song title and song's length as a result and there is a query condition that the sessionid should be 338 and itemInSession should be 4. 

In [10]:
try:
    query = " SELECT artist, song, length FROM song_playlist_session WHERE sessionId = 338 AND itemInSession  = 4; "
    rows = session.execute(query)
    if (rows.current_rows == []):
        print('Empty Set!')
    for row in rows:
        print(row.artist, row.song, row.length )
except Exception as e:
    print('ERROR!!')
    print(e)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


<br>

<br>

## Query 2

> Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

For this query,the combination of userid, sessionId, itemInSession is considered to be the composite key, because the query is based on attributes userid and sessionid and as there is a ordering conditionon itemInSession, it is added to the composite key. Here, userid acts as the partition key and sessionId and itemInSession are the clustering key. 
The attributes of the table are - ```userid, sessionId, itemInSession, fname, lname, artist, song```

In [11]:
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS user_playlist_session
    ( userid INT, sessionId INT,itemInSession INT,fname TEXT, lname TEXT, artist TEXT, song TEXT, PRIMARY KEY((userid,sessionId),itemInSession) );
    """)
except Exception as e:
    print('ERROR!!')
    print(e)        

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_playlist_session ( userid , sessionId ,itemInSession ,fname , lname , artist , song )"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]),int(line[3]), line[1], line[4], line[0], line[9]))

### SELECT Query

So according to the query, we require name of artist, song, user first and last name as a result, sorted by itemInSession and there is a query condition that the userid should be 10 and sessionid should be 182.
As itemInSession is a clustering column, so the results would automatically be sorted in order of itemInSession.

In [13]:
try:
    query = " SELECT artist, song, fname, lname  FROM user_playlist_session WHERE userid = 10 AND sessionId = 182; "
    rows = session.execute(query)
    if (rows.current_rows == []):
        print('Empty Set!')
    for row in rows:
        print(row.artist, row.song , row.fname,row.lname)
except Exception as e:
    print('ERROR!!')
    print(e)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


<br>

<br>

## Query 3

> Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

For this query,the combination of song and userid is considered to be the composite key, but the query is based on the song attribute, so why add userid to the key ? Because there can be many users listening to the same song and hence it would result in the overwriting of the result. Here, song acts as the partition key and userid as the clustering key. 
The attributes of the table are - ```song, userid, fname, lname```

In [14]:
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS songs_by_user
    ( song TEXT,userid INT, fname TEXT, lname TEXT, PRIMARY KEY(song,userid) );
    """)
except Exception as e:
    print('ERROR!!')
    print(e)

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_user ( song ,userid, fname , lname  )"
        query = query + "VALUES(%s,%s,%s,%s)"
        session.execute(query, (line[9],int(line[10]), line[1], line[4]))

### SELECT Query

So according to the query, we require the user's name (fname and lname) who listened to the song 'All Hands Against His Own'.
Hence we return the columns fname and lname, wth a condition that song = 'All Hands Against His Own' 

In [16]:
try:
    query = " SELECT fname, lname FROM songs_by_user WHERE song = 'All Hands Against His Own' ; "
    rows = session.execute(query)
    if (rows.current_rows == []):
        print('Empty Set!')
    for row in rows:
        print(row.fname, row.lname )
except Exception as e:
    print('ERROR!!')
    print(e)

Jacqueline Lynch
Tegan Levine
Sara Johnson


<br>

<br>

### Droping the table before closing out the sessions

In [17]:
try:
    session.execute("""    DROP TABLE song_playlist_session    """)
    session.execute("""    DROP TABLE user_playlist_session    """)
    session.execute("""    DROP TABLE songs_by_user    """)
except Exception as e:
    print('ERROR!!')
    print(e)

<br>

### Closing the session and cluster connection

In [18]:
session.shutdown()
cluster.shutdown()